In [1]:
from unsloth import FastLanguageModel
from unsloth import UnslothTrainer, UnslothTrainingArguments
from trl import SFTTrainer, SFTConfig
import torch

SEED = 42

/home/miguel/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 11-03 16:29:51 [__init__.py:216] Automatically detected platform cuda.
WARNING 11-03 16:29:51 [interface.py:391] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
model_name = "Qwen/Qwen3-8B"
MAX_LENGTH = 256

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = MAX_LENGTH,
    load_in_4bit = True,
    load_in_8bit = False,
)
tokenizer.pad_token = tokenizer.eos_token
RANK = 64
model = FastLanguageModel.get_peft_model(
    model,
    r = RANK,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = RANK*2,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = SEED,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

==((====))==  Unsloth 2025.9.8: Fast Qwen3 patching. Transformers: 4.56.2. vLLM: 0.10.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.24s/it]
Unsloth 2025.9.8 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


## Datasets

### Simple dataset loading example

In [3]:
import pandas as pd
from langchain.schema import Document
dataset_knowledge = pd.read_csv("../notebooks/data/contacts_docs.csv")
documents = []
for index, row in dataset_knowledge.iterrows():
    doc = f"Nombre: {row['name']}\nTeléfono: {row['phone']}"
    documents.append(Document(page_content=doc, metadata={"id": f"{row['id']}" } ))
print(f"Loaded {len(documents)} documents.")
print(f"First document: {documents[0]}")


Loaded 400 documents.
First document: page_content='Nombre: Alba Alonso
Teléfono: 632 322 183' metadata={'id': '7500_1'}


In [4]:
query_dataset_train = pd.read_csv("../notebooks/data/contacts_queries_train.csv")
query_dataset_val = pd.read_csv("../notebooks/data/contacts_queries_val.csv")
query_dataset_test = pd.read_csv("../notebooks/data/contacts_queries_test.csv")


In [5]:
all_data = {
    "train": query_dataset_train,
    "validation": query_dataset_val,
    "test": query_dataset_test,
}

#to hugginface dataset
from datasets import Dataset, DatasetDict
dataset_qa = {}
for split in all_data:
    dataset_qa[split] = Dataset.from_pandas(all_data[split])
dataset_qa = DatasetDict(dataset_qa)

In [6]:
dataset_qa

DatasetDict({
    train: Dataset({
        features: ['question', 'id', 'respuesta'],
        num_rows: 1400
    })
    validation: Dataset({
        features: ['question', 'id', 'respuesta'],
        num_rows: 300
    })
    test: Dataset({
        features: ['question', 'id', 'respuesta'],
        num_rows: 300
    })
})

In [7]:
model.print_trainable_parameters()

trainable params: 174,587,904 || all params: 8,365,323,264 || trainable%: 2.0870


In [8]:
dataset_qa

DatasetDict({
    train: Dataset({
        features: ['question', 'id', 'respuesta'],
        num_rows: 1400
    })
    validation: Dataset({
        features: ['question', 'id', 'respuesta'],
        num_rows: 300
    })
    test: Dataset({
        features: ['question', 'id', 'respuesta'],
        num_rows: 300
    })
})

## Data preparation

In [9]:
def build_prompt_it(tokenizer, system_prompt: str, prompt: str, response: str) -> str:
    """Builds the chat prompt for a single example using the tokenizer chat template."""
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": prompt},
        {"role": "assistant", "content": response}
    ]
    return tokenizer.apply_chat_template(
        messages,
        tokenize=False,
    )

In [10]:
def generate_knowledge_injection_prompts(documents: list):
    prompt = """{doc}"""
    for doc in documents:
        document = doc.page_content
        yield prompt.format(doc=document)

In [11]:
def generate_qa_prompts(dataset, tokenizer):
    system_prompt = """
    Eres un modelo de lenguaje entrenado para responder preguntas.
    """
    prompts = []
    for item in dataset:
        prompt = """{QUERY}"""
        response = "{response}"
        question = item["question"]
        prompt = prompt.format(QUERY=question)
        prompts.append(build_prompt_it(tokenizer, system_prompt, prompt, response.format(response=item["respuesta"])))
    return prompts

In [12]:
prompts = list(generate_knowledge_injection_prompts(documents))
print(f"Number of prompts: {len(prompts)}")

Number of prompts: 400


In [13]:
prompts[0]

'Nombre: Alba Alonso\nTeléfono: 632 322 183'

In [14]:
# QUIERO VER LOS TOKENS
def print_tokens(text):
    tokens = tokenizer.tokenize(text)
    print("Number of tokens:", len(tokens), "\n")
    print("Tokens:", tokens)

print_tokens(prompts[0])

Number of tokens: 21 

Tokens: ['Nombre', ':', 'ĠAl', 'ba', 'ĠAlonso', 'Ċ', 'Tel', 'Ã©fono', ':', 'Ġ', '6', '3', '2', 'Ġ', '3', '2', '2', 'Ġ', '1', '8', '3']


In [15]:
# create dataset from prompts
from datasets import Dataset
knowledge_dataset = Dataset.from_dict({"text": prompts})
knowledge_dataset

Dataset({
    features: ['text'],
    num_rows: 400
})

In [16]:
knowledge_dataset["text"][0]

'Nombre: Alba Alonso\nTeléfono: 632 322 183'

In [17]:
prompts_qa_train = generate_qa_prompts(dataset_qa["train"], tokenizer)
prompts_qa_val = generate_qa_prompts(dataset_qa["validation"], tokenizer)

print(f"Number of retrieval prompts: {len(prompts_qa_train)}")
print(f"Number of retrieval prompts: {len(prompts_qa_val)}")

Number of retrieval prompts: 1400
Number of retrieval prompts: 300


In [18]:
print(prompts_qa_train[0], sep="\n")

<|im_start|>system

    Eres un modelo de lenguaje entrenado para responder preguntas.
    <|im_end|>
<|im_start|>user
Necesito el contacto asociado al 620 152 344. —consulta interna—<|im_end|>
<|im_start|>assistant
<think>

</think>

El número 620 152 344 pertenece a Alejandro Vega.<|im_end|>



In [19]:
print_tokens(prompts_qa_train[0])

Number of tokens: 84 

Tokens: ['<|im_start|>', 'system', 'ĊĊ', 'ĠĠĠ', 'ĠE', 'res', 'Ġun', 'Ġmodelo', 'Ġde', 'Ġl', 'engu', 'aje', 'Ġentren', 'ado', 'Ġpara', 'Ġresponder', 'Ġpreg', 'untas', '.Ċ', 'ĠĠĠĠ', '<|im_end|>', 'Ċ', '<|im_start|>', 'user', 'Ċ', 'N', 'ec', 'es', 'ito', 'Ġel', 'Ġcontacto', 'Ġasoci', 'ado', 'Ġal', 'Ġ', '6', '2', '0', 'Ġ', '1', '5', '2', 'Ġ', '3', '4', '4', '.', 'ĠâĢĶ', 'consulta', 'Ġintern', 'a', 'âĢĶ', '<|im_end|>', 'Ċ', '<|im_start|>', 'assistant', 'Ċ', '<think>', 'ĊĊ', '</think>', 'ĊĊ', 'El', 'ĠnÃºmero', 'Ġ', '6', '2', '0', 'Ġ', '1', '5', '2', 'Ġ', '3', '4', '4', 'Ġpert', 'ene', 'ce', 'Ġa', 'ĠAlejandro', 'ĠVega', '.', '<|im_end|>', 'Ċ']


In [20]:
# create dataset from prompts train, val, test
qa_train_dataset = Dataset.from_dict({"text": prompts_qa_train})
qa_val_dataset = Dataset.from_dict({"text": prompts_qa_val})

qa_dataset = {
    "train": qa_train_dataset,
    "validation": qa_val_dataset,
}

In [21]:
def tokenize_function_autoregressive(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=MAX_LENGTH)

In [22]:
knowledge_dataset_tokenizer = knowledge_dataset.map(tokenize_function_autoregressive, batched=True)

Map: 100%|██████████| 400/400 [00:00<00:00, 14314.18 examples/s]


In [23]:
qa_train_dataset_tokenizer = qa_dataset["train"].map(tokenize_function_autoregressive, batched=True)
qa_val_dataset_tokenizer = qa_dataset["validation"].map(tokenize_function_autoregressive, batched=True)

Map: 100%|██████████| 300/300 [00:00<00:00, 16129.25 examples/s]


## Train

In [24]:
# sft training
from transformers import DataCollatorForLanguageModeling


data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
auto_config = UnslothTrainingArguments(
    per_device_train_batch_size = 8,
    gradient_accumulation_steps = 8, # Use GA to mimic batch size!
    save_steps=5,
    warmup_steps = 5,
    num_train_epochs = 1, # Set this for 1 full training run.
    #max_steps = 60,
    learning_rate = 1e-4, # Reduce to 2e-5 for long training runs
    logging_steps = 1,
    # 32 bits
    optim = "paged_adamw_32bit",
    weight_decay = 0.01,
    lr_scheduler_type = "cosine",
    seed = SEED,
    report_to = "none", # Use this for WandB etc
    output_dir="../models/qwen3-0.6b-rag-indexer",
)

it_config = SFTConfig(
    dataset_text_field="text",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,         # <-- añade eval batch size
    gradient_accumulation_steps=16,
    warmup_steps=25,
    save_steps=25,
    eval_steps=1,
    eval_strategy="steps",         # <-- activa evaluación periódica
    num_train_epochs=1,             # <-- opcional: usa epochs en lugar de max_steps
    #max_steps=30,
    learning_rate=1e-4,
    logging_steps=1,
    optim = "paged_adamw_32bit",
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    seed=SEED,
    report_to="none",
    output_dir="../models/qwen3-0.6b-rag-retriever",
    load_best_model_at_end=True,          # <-- opcional
    metric_for_best_model="eval_loss",    # <-- opcional
    greater_is_better=False,              # <-- opcional
)

trainer_auto = UnslothTrainer(
    model=model,
    train_dataset=knowledge_dataset_tokenizer,
    tokenizer=tokenizer,
    data_collator=data_collator,
    args=auto_config,
)

trainer_it = SFTTrainer(
    model=model,
    train_dataset=qa_train_dataset_tokenizer,
    eval_dataset=qa_val_dataset_tokenizer,
    data_collator=data_collator,
    tokenizer=tokenizer,
    args=it_config,
)

In [25]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.988 GB.
7.711 GB of memory reserved.


In [26]:
model.print_trainable_parameters()

trainable params: 174,587,904 || all params: 8,365,323,264 || trainable%: 2.0870


In [27]:
EPOCHS = 10
for _ in range(EPOCHS):
    print(f"--- SUPER EPOCH {_+1} / {EPOCHS} ---")
    trainer_sft_stats = trainer_auto.train() 
    trainer_it_stats = trainer_it.train()
    # GUARDAR MODELOS CADA SUPER EPOCH

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.


--- SUPER EPOCH 1 / 10 ---


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 400 | Num Epochs = 1 | Total steps = 7
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 8 x 1) = 64
 "-____-"     Trainable parameters = 174,587,904 of 8,365,323,264 (2.09% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.420700
2,2.383800
3,2.364600
4,2.110200
5,1.807500
6,1.640200
7,1.535700


...

--- SUPER EPOCH 10 / 10 ---


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 400 | Num Epochs = 1 | Total steps = 7
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 8 x 1) = 64
 "-____-"     Trainable parameters = 174,587,904 of 8,365,323,264 (2.09% trained)


Step,Training Loss
1,1.226800
2,1.212300
3,1.154300
4,1.094900
5,1.091300
6,1.125700
7,0.895100


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,400 | Num Epochs = 1 | Total steps = 6
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 16 x 1) = 256
 "-____-"     Trainable parameters = 174,587,904 of 8,365,323,264 (2.09% trained)


Step,Training Loss,Validation Loss
1,0.486900,0.497809
2,0.498000,0.493198
3,0.488700,0.483497
4,0.481900,0.470966
5,0.464100,0.461107
6,0.464000,0.450997


In [28]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

Peak reserved memory = 9.961 GB.
Peak reserved memory for training = 2.25 GB.
Peak reserved memory % of max memory = 41.525 %.
Peak reserved memory for training % of max memory = 9.38 %.


In [29]:
def generate_qa_prompts_testing(dataset, tokenizer):
    def build_prompt_it_generation(tokenizer, system_prompt: str, prompt: str) -> str:
        """Builds the chat prompt for a single example using the tokenizer chat template."""
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user",   "content": prompt},
        ]
        return tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=False,
        )
    system_prompt = """
    Eres un modelo de lenguaje entrenado para responder preguntas.
    """
    prompts = []
    for item in dataset:
        prompt = """{QUERY}"""
        question = item["question"]
        answer = item["respuesta"]
        prompt = prompt.format(QUERY=question)
        prompts.append(
            (
                build_prompt_it_generation(tokenizer, system_prompt, prompt),
                answer,
            )
        )
    return prompts

In [30]:
prompts_retrieval_test = generate_qa_prompts_testing(dataset_qa["test"], tokenizer)

In [31]:
idx = 0
print(prompts_retrieval_test[idx][0], sep="\n")
print(prompts_retrieval_test[idx][1], sep="\n")
text = prompts_retrieval_test[idx][0]

<|im_start|>system

    Eres un modelo de lenguaje entrenado para responder preguntas.
    <|im_end|>
<|im_start|>user
¿A quién pertenece el teléfono 736 615 948?<|im_end|>
<|im_start|>assistant

El número 736 615 948 pertenece a Javier Martín.


In [32]:
# test the model in streaming mode
from transformers import TextStreamer

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=False)
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    do_sample = False,
    top_p = 0.1,
    temperature = 0.,
    streamer = streamer,
)

<think>

</think>

El número 736 615 948 pertenece a Nerea Díaz.<|im_end|>


In [33]:
# test the model in non-streaming mode
import re
import tqdm

acc = 0
total = 0

for text, answer in tqdm.tqdm(prompts_retrieval_test, desc="Testing"):
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=64,  # Increase for longer outputs!
        do_sample=False, temperature=0.0, top_p=1.0
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
    response = generated_text.split("</think>")[-1]
    print(response)
    # extract DOCID number using regex
    
    if response == answer:
        print("Correct:", answer, "==", "Predicted:", response)
        acc += 1
    total += 1
print(f"Accuracy: {acc}/{total} = {acc/total*100:.2f} %")

Testing:   0%|          | 1/300 [00:01<06:47,  1.36s/it]



El número 736 615 948 pertenece a Nerea Díaz.<|im_end|>


Testing:   1%|          | 2/300 [00:02<06:40,  1.34s/it]



El número de contacto de Manuel Sánchez es 610 470 555.<|im_end|>


Testing:   1%|          | 3/300 [00:04<06:49,  1.38s/it]

...



El número 735 202 200 pertenece a Nerea Díaz.<|im_end|>


Testing: 100%|█████████▉| 299/300 [08:05<00:01,  1.39s/it]



El número de contacto de María Romero es 606 841 222.<|im_end|>


Testing: 100%|██████████| 300/300 [08:06<00:00,  1.62s/it]



El número de contacto de Leyre Romero es 636 211 678.<|im_end|>
Accuracy: 0/300 = 0.00 %
